## 60分钟MA5&MA13

In [ ]:
# 60分钟一股MA5&ma13y
# 60分钟MA5&MA13

# 初始化函数,全局只运行一次
import matplotlib.pyplot as plt
import datetime

def init(context):
    # 打印日志
    print('策略开始运行,初始化函数全局只运行一次')
    # 设置股票每笔交易的手续费
    set_commission(PerShare(type='stock',cost=0.00025))
    # 设置基准收益
    #set_benchmark('000300.SH')
    set_benchmark('002060.SZ')
    # 设置操作标的：
    #context.stockpool = ['002545.SZ','002941.SZ','600975.SH','600153.SH','600358.SH','000893.SZ','600985.SH','600716.SH','002567.SZ','002625.SZ', '600657.SH', '300803.SZ', '002060.SZ', '688208.SH', '600359.SH']
    #context.security = ['002625.SZ', '600657.SH', '300803.SZ', '002060.SZ', '688208.SH', '600359.SH']
    context.stockpool = ['002060.SZ']
    
    # 单个股票的仓位
    # context.buypct = 1/len(context.stockpool)
    context.buypct = 1
    # 止盈
    #context.stopwin = 888
    # 止损
    context.stoploss = 0.3
    
    # 记录时间
    # 获取时间
    time1 = get_datetime().strftime('%Y-%m-%d %H:%M:%S')
    context.date1 = time1[:10]
    context.date = time1[:10]
    context.runtime = 0
    print('context.runtime:', type(context.runtime), context.runtime)

    # 设置初识持仓
    set_holding_stocks(dict(zip(context.stockpool,[40000]*len(context.stockpool))))
    market_value = context.portfolio.stock_account.market_value
    total_value = context.portfolio.stock_account.total_value
    stocklist = list(context.portfolio.stock_account.positions)
    print('当前持仓市值:', market_value, '持仓股票列表', stocklist, '当前账户总资金:', int(total_value))
    print()
 
        
# 盘中运行函数
def handle_bar(context, bar_dict):
    # 导入Python-talib库
    import talib
    

    # 数据获取与处理模块
    # 获取当前持仓
    context.runtime += 1
    holdstock = list(context.portfolio.stock_account.positions.keys())
    #print('当前持仓holdstock:', holdstock)
    for stock in context.stockpool:
        # 信号识别与下单模块
        # 获取股票过去15个小时的分钟收盘价数据
        closeprice = history(stock, ['close'], 900, '1m', False, 'pre', is_panel=1)
        time = get_datetime().strftime('%Y-%m-%d %H:%M:%S')

        # 计算小时K线的MA13均线
        MA13 = closeprice['close'].iloc[-780:-1].mean()
        MA13a = closeprice['close'].iloc[-840:-61].mean()
        # 计算MA5均线
        MA5 = closeprice['close'].iloc[-300:-1].mean()
        MA5a = closeprice['close'].iloc[-360:-61].mean()

        # 记录

        
        if context.runtime%60==0:
            #print('stock:',stock,'MA5=',MA5,'MA13=',MA13)
            # 如果MA5均线下窜死叉MA13均线,且账户当前有持仓,则该股票按占比context.buypct卖出
            if MA5a>MA13a and MA5<MA13 and stock in holdstock:
                new_cost = closeprice['close'].iloc[-1]
                cost_basis = context.portfolio.positions[stock].cost_basis
                # 记录这次卖出
                order_percent(stock, -context.buypct)
                #order_value(stock, -0.2)
                print('MA5=',MA5,'MA13=',MA13)
                log.info("MA5均线下窜死叉MA13均线, 卖出context.buypct占比 %s" % stock, '盈亏:',int((new_cost-cost_basis)/cost_basis*10000)/100,'%','成本价=',int(cost_basis*100)/100)

                market_value = context.portfolio.stock_account.market_value
                total_value = context.portfolio.stock_account.total_value
                stocklist = list(context.portfolio.stock_account.positions)
                print('当前持仓市值:', market_value, '持仓股票列表', stocklist, '当前账户总资金:', int(total_value))
                print()


            # 如果MA5均线上窜金叉MA13均线,则按仓位占比context.buypct买入股票
            if MA5a < MA13a and MA5 > MA13 :
                # 记录这次买入
                order_percent(stock, context.buypct)
                cost_basis = context.portfolio.positions[stock].cost_basis
                print('MA5=',MA5,'MA13=',MA13)
                log.info("MA5均线上窜金叉MA13均线, 买入context.buypct占比 %s" % stock, '成本价=',int(cost_basis*100)/100)
                #order_target_percent(stock, 0.2)
                #order_value(stock, 0.2)

                market_value = context.portfolio.stock_account.market_value
                total_value = context.portfolio.stock_account.total_value
                stocklist = list(context.portfolio.stock_account.positions)
                print('当前持仓市值:', market_value, '持仓股票列表', stocklist, '当前账户总资金:', int(total_value))
                print()
          


# 收盘后运行函数,用于储存自定义参数、全局变量,执行盘后选股等
def after_trading(context):
    # market_value = context.portfolio.stock_account.market_value
    # total_value = context.portfolio.stock_account.total_value
    # stocklist = list(context.portfolio.stock_account.positions)
    # print('当前持仓市值:', market_value, '持仓股票列表', stocklist, '当前账户总资金:', int(total_value))
    
    
    # 获取时间
    time = get_datetime().strftime('%Y-%m-%d %H:%M:%S')
    # 打印时间
    log.info('{} 盘后运行'.format(time))
    log.info('一天结束')
    




